In [1]:
# Dependencies
import pandas as pd
import pymongo
import time
import datetime as dt
import numpy as np

In [2]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
db = client.housing_db
collection = db.redfinclean
df = pd.DataFrame(list(db.redfin.find().limit(10000)))

In [3]:
sel_cols = ['period_begin',  
'period_end', 
'region_name' ,
'region_type',
'total_homes_sold',
'median_sale_price',
'median_sale_ppsf'
]

In [4]:
df_reduced = df[sel_cols]

In [5]:
# df_reduced["period_begin"].to_datetime()
# 
begin_dates = pd.to_datetime(df_reduced["period_begin"], infer_datetime_format=True)
df_reduced["year"] = begin_dates.dt.strftime("%Y")

df_reduced["year_month"] = begin_dates.dt.strftime("%Y-%m")

states = []
for region in df_reduced["region_name"]:
    reg_list = region.split(",")
    if len(reg_list) == 2:
        reg_state = reg_list[1][1:3]
        states.append(reg_list[1][1:3])
    else:
        states.append("")
df_reduced["state"] = states

/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_index

In [6]:
df_nometro = df_reduced.loc[df_reduced["region_type"] != "metro"]

In [7]:
df_nometro.head()

,period_begin,period_end,region_name,region_type,total_homes_sold,median_sale_price,median_sale_ppsf,year,year_month,state
0,2019-09-09,2019-09-15,"Haywood County, NC",county,34.0,240000.0,143.20260577745609,2019,2019-09,NC
1,2018-11-05,2018-11-11,"Eaton County, MI",county,19.0,187500.0,121.765601217656,2018,2018-11,MI
2,2020-07-20,2020-07-26,"Greenville County, SC",county,207.0,255000.0,121.04283054003724,2020,2020-07,SC
3,2019-02-04,2019-02-10,"Dane County, WI",county,65.0,274000.0,147.58389261744966,2019,2019-02,WI
4,2017-05-01,2017-05-07,"Aroostook County, ME",county,8.0,58700.0,45.265126771662885,2017,2017-05,ME


In [8]:
df_nometro["region_type"].value_counts()


county    7921
Name: region_type, dtype: int64

In [9]:

df_nometro.fillna('', inplace=True)
df_nometro.head()


/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/pandas/core/frame.py:4153: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


,period_begin,period_end,region_name,region_type,total_homes_sold,median_sale_price,median_sale_ppsf,year,year_month,state
0,2019-09-09,2019-09-15,"Haywood County, NC",county,34.0,240000.0,143.20260577745609,2019,2019-09,NC
1,2018-11-05,2018-11-11,"Eaton County, MI",county,19.0,187500.0,121.765601217656,2018,2018-11,MI
2,2020-07-20,2020-07-26,"Greenville County, SC",county,207.0,255000.0,121.04283054003724,2020,2020-07,SC
3,2019-02-04,2019-02-10,"Dane County, WI",county,65.0,274000.0,147.58389261744966,2019,2019-02,WI
4,2017-05-01,2017-05-07,"Aroostook County, ME",county,8.0,58700.0,45.265126771662885,2017,2017-05,ME


In [10]:
final_cols = ['year',
              'state',
'total_homes_sold',
'median_sale_price',
]

In [11]:
df_redfinclean = df_nometro[final_cols]
df_redfinclean.head()

,year,state,total_homes_sold,median_sale_price
0,2019,NC,34.0,240000.0
1,2018,MI,19.0,187500.0
2,2020,SC,207.0,255000.0
3,2019,WI,65.0,274000.0
4,2017,ME,8.0,58700.0


In [12]:
df_redfinclean["year"].value_counts()

2018    1947
2019    1847
2020    1806
2017    1758
2021     563
Name: year, dtype: int64

In [13]:
df_redfinclean["median_sale_price"].dtype

dtype('O')

In [14]:
df_redfinclean["total_homes_sold"].dtype

dtype('O')

In [15]:
df_redfinclean["median_sale_price"] = pd.to_numeric(df_redfinclean["median_sale_price"])
df_redfinclean["median_sale_price"].dtype

/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


dtype('float64')

In [16]:
df_redfinclean["total_homes_sold"] = pd.to_numeric(df_redfinclean["total_homes_sold"])
df_redfinclean["median_sale_price"].dtype

/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


dtype('float64')

In [17]:
df_redfinclean

,year,state,total_homes_sold,median_sale_price
0,2019,NC,34.0,240000.0
1,2018,MI,19.0,187500.0
2,2020,SC,207.0,255000.0
3,2019,WI,65.0,274000.0
4,2017,ME,8.0,58700.0
...,...,...,...,...
9994,2017,NH,22.0,163000.0
9996,2017,KY,3.0,55320.0
9997,2020,FL,364.0,216000.0
9998,2020,NJ,44.0,560500.0


In [18]:
state_list = df_redfinclean.state.unique().tolist()
state_list

['NC',
 'MI',
 'SC',
 'WI',
 'ME',
 'PA',
 'MO',
 'MD',
 'MN',
 'NY',
 'FL',
 'TN',
 'OK',
 'KY',
 'AL',
 'VA',
 'TX',
 'AZ',
 'IL',
 'NE',
 'CO',
 'UT',
 'ID',
 'WV',
 'LA',
 'IN',
 'AR',
 'MA',
 'WA',
 'OR',
 'GA',
 'OH',
 'NH',
 'IA',
 'NV',
 'CA',
 'KS',
 'CT',
 'AK',
 'NJ',
 'HI',
 'VT',
 'DE',
 'NM',
 'RI',
 'MS',
 'DC']

In [19]:
redfinstatesum = df_redfinclean.groupby(['state', 'year']).agg({'total_homes_sold': np.sum, 'median_sale_price': np.average})
redfinstatesum.reset_index(inplace = True)
redfinstatesum.head()

,state,year,total_homes_sold,median_sale_price
0,AK,2017,95.0,256290.0
1,AK,2018,261.0,273500.0
2,AK,2019,240.0,277600.0
3,AK,2020,263.0,326750.0
4,AK,2021,87.0,369000.0


In [20]:
redfinstatesum.fillna('', inplace=True)

In [21]:
redfin_dict = redfinstatesum.to_dict('records')
redfin_dict

[{'state': 'AK',
  'year': '2017',
  'total_homes_sold': 95.0,
  'median_sale_price': 256290.0},
 {'state': 'AK',
  'year': '2018',
  'total_homes_sold': 261.0,
  'median_sale_price': 273500.0},
 {'state': 'AK',
  'year': '2019',
  'total_homes_sold': 240.0,
  'median_sale_price': 277600.0},
 {'state': 'AK',
  'year': '2020',
  'total_homes_sold': 263.0,
  'median_sale_price': 326750.0},
 {'state': 'AK',
  'year': '2021',
  'total_homes_sold': 87.0,
  'median_sale_price': 369000.0},
 {'state': 'AL',
  'year': '2017',
  'total_homes_sold': 1252.0,
  'median_sale_price': ''},
 {'state': 'AL',
  'year': '2018',
  'total_homes_sold': 1931.0,
  'median_sale_price': ''},
 {'state': 'AL',
  'year': '2019',
  'total_homes_sold': 1889.0,
  'median_sale_price': ''},
 {'state': 'AL',
  'year': '2020',
  'total_homes_sold': 1619.0,
  'median_sale_price': ''},
 {'state': 'AL',
  'year': '2021',
  'total_homes_sold': 285.0,
  'median_sale_price': 267340.75},
 {'state': 'AR',
  'year': '2017',
  'tot

In [22]:

#dictionary to hold output


redfin_output_dictionary = {
    "data": redfin_dict,
    "states": state_list
}

redfin_output_dictionary

{'data': [{'state': 'AK',
   'year': '2017',
   'total_homes_sold': 95.0,
   'median_sale_price': 256290.0},
  {'state': 'AK',
   'year': '2018',
   'total_homes_sold': 261.0,
   'median_sale_price': 273500.0},
  {'state': 'AK',
   'year': '2019',
   'total_homes_sold': 240.0,
   'median_sale_price': 277600.0},
  {'state': 'AK',
   'year': '2020',
   'total_homes_sold': 263.0,
   'median_sale_price': 326750.0},
  {'state': 'AK',
   'year': '2021',
   'total_homes_sold': 87.0,
   'median_sale_price': 369000.0},
  {'state': 'AL',
   'year': '2017',
   'total_homes_sold': 1252.0,
   'median_sale_price': ''},
  {'state': 'AL',
   'year': '2018',
   'total_homes_sold': 1931.0,
   'median_sale_price': ''},
  {'state': 'AL',
   'year': '2019',
   'total_homes_sold': 1889.0,
   'median_sale_price': ''},
  {'state': 'AL',
   'year': '2020',
   'total_homes_sold': 1619.0,
   'median_sale_price': ''},
  {'state': 'AL',
   'year': '2021',
   'total_homes_sold': 285.0,
   'median_sale_price': 26734

In [23]:
collection.insert_one(redfin_output_dictionary)